In [123]:
import json, pprint, requests, textwrap, time
host = 'http://150.140.142.92:8998'

In [124]:
#get number of sessions
headers = {'Content-Type': 'application/json'}
r = requests.get(host+'/sessions', headers=headers)
con_var=r.json()['total']
print ("Waiting... active sesssions: ", con_var)
while con_var!=0:
    r = requests.get(host+'/sessions', headers=headers)
    con_var=r.json()['total']
    print ("Waiting... active sesssions: ", con_var)
    time.sleep(60)
print ("Good to go!")

Waiting... active sesssions:  0
Good to go!


In [125]:
sessionid = 285
r = requests.delete("http://150.140.142.92:8998/sessions/"+str(sessionid), headers={'Content-Type': 'application/json'})
pprint.pprint(r.json())

{'msg': "Session '285' not found."}


In [126]:
data = {'kind': 'pyspark',\
        'name':'QueriesforLivy.py',
        'conf': {'master':'spark://administrator-OptiPlex-390:7077'}}

r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 298,
 'name': 'query1',
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [127]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 298,
 'name': 'query1',
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [128]:
#show session state, repeat until idle to proceed with result extraction
state_url = session_url + '/state'
con_var=".."
while con_var!="idle":
    r = requests.get(state_url, headers=headers)
    con_var = r.json()['state']
    print("Job is %s" % r.json()['state'])
    time.sleep(10)
print ("Good to go!")

Job is starting
Job is idle
Good to go!


In [129]:
#send some python statements for execution
codefile = open('query1.py', 'r')
code = codefile.read()
codefile.close()
data = {
    'kind': 'pyspark',
    'code': textwrap.dedent(code)
}

statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)

In [130]:
#get stats from the latest statement in the session
statements_url = session_url + '/statements'
print(statements_url)
r = requests.get(statements_url, headers=headers)
slist = r.json()
last_statement = slist['statements'][slist['total_statements']-1]
if last_statement['progress']==1:
    print ("Job completed")

print("Total Time %s ms" % (last_statement['completed'] - last_statement['started']))

http://150.140.142.92:8998/sessions/298/statements
Total Time 0 ms


In [122]:
r = requests.delete(session_url, headers=headers)
pprint.pprint(r.json())

{'msg': 'deleted'}
